Here we try to reproduce the `Model/synthNetDataScreen.py` script from Avlant's work


It was run on the cluster. The args were found in the `sunSlurmSynthScreen.sh` file. It takes a value between 0 and 14. 



In [1]:
import sys
sys.path.insert(0, '../')
from nn_cno import nn_models
import numpy as np
import scipy as sp
import pandas as pd
import equinox as eqx
import optax
import jax.tree_util as jtu
from jax.experimental import sparse
from jax import numpy as jnp
import jax
import matplotlib.pyplot as plt
import functools as ft

# import argparse
# This code is used to evaluate the different parts on a cluster. 

# #Get data number
# parser = argparse.ArgumentParser(prog='Macrophage simulation')
# parser.add_argument('--selectedCondition', action='store', default=None)
# args = parser.parse_args()
# curentId = int(args.selectedCondition)

currentID = 0     # goes between 0-14 based on the sunSlurmSynthScreen.sh file. 

Initialization of the model using the new implementation:

In [12]:

testCondtions = pd.read_csv('synthNetScreen/conditions.tsv', sep='\t', low_memory=False)
simultaniousInput = int(testCondtions.loc[currentID == testCondtions['Index'],:]['Ligands'].values)
N = int(testCondtions.loc[currentID == testCondtions['Index'],:]['DataSize'].values)
print(currentID, simultaniousInput, N)

inputAmplitude = 3
projectionAmplitude = 1.2

modelFile = "data/KEGGnet-Model.tsv"
annotationFile = 'data/KEGGnet-Annotation.tsv'
parameterFile = 'synthNetScreen/equationParams.txt'

parameterizedModel = nn_models.bioNetwork(networkFile=modelFile, 
                                          nodeAnnotationFile=annotationFile,
                                          inputAmplitude=inputAmplitude,
                                          projectionAmplitude=projectionAmplitude)
parameterizedModel.loadParams(parameterFile)

Model  = nn_models.bioNetwork(networkFile=modelFile, 
                                          nodeAnnotationFile=annotationFile,
                                          inputAmplitude=inputAmplitude,
                                          projectionAmplitude=projectionAmplitude)

0 2 10


In-silico data was generated by simulating the model with given parameters.
I think the in-silico model was a result of some kind of optimization, because the paper
mentions that the random parameterization results in pretty uniform output. 


In [13]:
#Generate data
X = np.zeros((N, len(parameterizedModel.network.inName)))
for i in range(1, N): #skip 0 to include a ctrl sample i.e. zero input
    X[i, (i-1) % len(parameterizedModel.network.inName)] = np.random.rand(1) #stimulate each receptor at least once
    X[i, np.random.randint(0, len(parameterizedModel.network.inName), simultaniousInput-1)] = np.random.rand(simultaniousInput-1)
# compared X with pytorch version

controlIndex = 0
# Y, YfullRef = parameterizedModel(X)
Y, YfullRef = jax.vmap(parameterizedModel.model, in_axes=(0),out_axes=(0,0))(X)


I used the `dataloader_inf` for the toy model. It reuses the training data if the batch `size * n_steps` is larger than the available training data. 

but we should use the `dataloader_finite` for the comparison with Avlant's approach. This use each data only once. 

In [14]:
# pytorch has a data loader, but jax does not.
# trainloader = torch.utils.data.DataLoader(range(N), batch_size=batchSize, shuffle=True)

# here we take the example from https://docs.kidger.site/equinox/examples/train_rnn/
# from equinox

# this is another version of dataloader, which can be emptied. 
import jax.random as jrandom

# 
def dataloader_inf(arrays, batch_size, *, key):
    """ Create an infinite dataloader from data. 
    
    It goes through the data and returns batches of size batch_size.
    When it finishes, it starts again from the beginning.

    Args:
        arrays: list of arrays to be batched.
        batch_size: size of the batch.
        key: random key.
    """
    dataset_size = arrays[0].shape[0]
    assert all(array.shape[0] == dataset_size for array in arrays)
    indices = jnp.arange(dataset_size)
    while True:
        perm = jrandom.permutation(key, indices)
        (key,) = jrandom.split(key, 1)
        start = 0
        end = batch_size
        while end < dataset_size:
            batch_perm = perm[start:end]
            yield tuple(array[batch_perm] for array in arrays)
            start = end
            end = start + batch_size

def dataloader_finite(arrays, batch_size, *, key):
    """ Create an finite dataloader from data. 
    
    It goes through the data and returns batches of size batch_size, finally, 
    it returns the last batch if the data is not divisible by batch_size.

    Args:
        arrays: list of arrays to be batched.
        batch_size: size of the batch.
        key: random key.
    """
    dataset_size = arrays[0].shape[0]
    assert all(array.shape[0] == dataset_size for array in arrays)
    indices = jnp.arange(dataset_size)

    perm = jrandom.permutation(key, indices)
    (key,) = jrandom.split(key, 1)
    start = 0
    end = batch_size
    while end < dataset_size:
        batch_perm = perm[start:end]
        yield tuple(array[batch_perm] for array in arrays)
        start = end
        end = start + batch_size
    
    if start < dataset_size:
        batch_perm = perm[start:]
        yield tuple(array[batch_perm] for array in arrays)


def dataIndexloader(arrays, batch_size, *, key):
    """ Create the indices used in the batch.
    
    Args:
        arrays: list of arrays to be batched.
        batch_size: size of the batch.
        key: random key.
    """
    dataset_size = arrays[0].shape[0]
    assert all(array.shape[0] == dataset_size for array in arrays)
    indices = jnp.arange(dataset_size)

    perm = jrandom.permutation(key, indices)
    (key,) = jrandom.split(key, 1)
    start = 0
    end = batch_size
    while end < dataset_size:
        batch_perm = perm[start:end]
        yield batch_perm
        start = end
        end = start + batch_size
    
    if start < dataset_size:
        batch_perm = perm[start:]
        yield batch_perm


## Check how to rewrite the `uniformLoss` 

In [15]:
#@jax.jit
def uniformLoss(curState, dataIndex, YhatFull, targetMin = 0, targetMax = 0.99, maxConstraintFactor = 10):
    #data = curState.detach().clone()
    #data[dataIndex, :] = YhatFull
    loss = uniformLossBatch(curState, targetMin = targetMin, targetMax = targetMax, maxConstraintFactor = maxConstraintFactor)
    return loss

#@jax.jit
def uniformLossBatch(YhatFull, targetMin = 0, targetMax = 0.99, maxConstraintFactor = 10):
    targetMean = (targetMax-targetMin)/2
    targetVar = (targetMax-targetMin)**2/12

    factor = 1
    meanFactor = factor
    varFactor = factor
    minFactor = factor
    maxFactor = factor
    maxConstraintFactor = factor * maxConstraintFactor

    nodeMean = jnp.mean(YhatFull, axis=0)
    nodeVar = jnp.mean(jnp.square(YhatFull-nodeMean), axis=0)
    maxVal = jnp.amax(YhatFull, axis=0)
    minVal = jnp.amin(YhatFull, axis=0)

    meanLoss = meanFactor * jnp.sum(jnp.square(nodeMean - targetMean))
    varLoss =  varFactor * jnp.sum(jnp.square(nodeVar - targetVar))
    maxLoss = maxFactor * jnp.sum(jnp.square(maxVal - targetMax))
    minloss = minFactor * jnp.sum(jnp.square(minVal- targetMin))
    #maxConstraint = -maxConstraintFactor * jnp.sum(maxVal[maxVal<=0]) #max value should never be negative
    maxConstraint = -maxConstraintFactor * jnp.sum(jnp.where(maxVal<=0, maxVal, 0))
     
    loss = meanLoss + varLoss + minloss + maxLoss + maxConstraint
    return loss



testing `uniformLoss`

In [16]:
batch_size = 5
#maxIter = 100
loader_key = jax.random.PRNGKey(142)
#progress = nn_models.rnnModel.OptimProgress(maxIter)


In [17]:
# batch_size = 5
# loader_key = jax.random.PRNGKey(142)
# iter_data = dataIndexloader((X, Y), batch_size, key=loader_key)
# dataIndex = next(iter_data)
# x = X[dataIndex, :]
# y = Y[dataIndex, :]
# print(x.shape)
# (pred_y, pred_yFull) = jax.vmap(Model.model, out_axes=(0,0))(x)
# #currentState = jax.random.uniform(shape=(N, Model.model.layers[1].biases.shape[0]),key=statekey)
# currentState = jnp.zeros(shape=(N, 1, Model.model.layers[1].biases.shape[0]))

# print(currentState.shape)
# print(pred_yFull.shape)
# stateLossFactor = 1e-4

# currentState = currentState.at[dataIndex,1,:].set(jnp.squeeze(pred_yFull))

# # State constraints: 
# stateLoss = stateLossFactor * uniformLoss(currentState, dataIndex, pred_yFull, maxConstraintFactor = 50)
# stateLoss


## Checking spectral loss computation

In [18]:
# @jax.jit
def oneStepDeltaActivationFactor(x, leak):
    """ derivative of the activation function F(x):
        x < 0:        leak 
        0 < x < 0.5 : 1
        0.5 < x :      0.25/x^2
    """
    y = jnp.ones(x.shape[1]) #derivative = 1 if nothing else is stated
    y = jnp.where(x <= 0, leak, y)  #let derivative be 0.01 at x=0
    
    y = jnp.where(x > 0.5, 0.25/(x**2), y)
    return y

def lreig(A):
    # for DENSE Matrix using scipy
    #fall back if eigs fails
    e, w, v = sp.linalg.eig(A, left = True)
    
    selected = np.argmax(np.abs(e))
    eValue = e[selected]
    w = w[:,selected]
    v = v[:,selected]
    
    return eValue, v, w

# A: sparse matrix
@jax.custom_jvp
def getspectralRadius(weights, ind, M):
    
    A = sp.sparse.csr_matrix((weights, ind), shape=M.shape, dtype='float32')
    tolerance = 10**-6

    try:
        e, v = sp.sparse.linalg.eigs(A, k=1, which='LM', ncv=100, tol = tolerance)
        v = v[:,0]
        e = e[0]
    except  (KeyboardInterrupt, SystemExit):
        raise
    except:
        print('Forward failed in forward (did not find any eigenvalue with eigs)')
        tmpA = A.toarray()
        e, v, w = lreig(tmpA) #fall back to solving full eig problem

    spectralRadius = np.abs(e)

    return spectralRadius

@getspectralRadius.defjvp
def AgetspectralRadius_jvp(primals, tangents):
    weights, ind, M = primals
    w_dot, ind_dot, M_dot  = tangents

    networkList = ind
    A = sp.sparse.csr_matrix((weights, ind), shape=M.shape, dtype='float32')
    tolerance = 10**-6
    w = np.empty(0)

    try:
        e, v = sp.sparse.linalg.eigs(A, k=1, which='LM', ncv=100, tol = tolerance)
        v = v[:,0]
        e = e[0]
    except  (KeyboardInterrupt, SystemExit):
        raise
    except:
        print('Forward failed in backward (did not find any eigenvalue with eigs)')
        tmpA = A.toarray()
        e, v, w = lreig(tmpA) #fall back to solving full eig problem

    primal_out = np.abs(e)
    
    tmpA = A.T 

    if w.shape[0]==0:
        try:
            eT = e
            if np.isreal(eT): #does for some reason not converge if imag = 0
                eT = eT.real
            e2, w = sp.sparse.linalg.eigs(tmpA, k=1, sigma=eT, OPpart='r', tol=tolerance)
            selected = 0 #numpy.argmin(numpy.abs(e2-eT))
            w = w[:,selected]
            e2 = e2[selected]
            #Check if same eigenvalue
            if abs(e-e2)>(tolerance*10):
                print('Backward fail (eigs left returned different eigenvalue)')
                w = np.empty(0)
                #e, v, w = lreig(tmpA) #fall back to solving whole eig problem
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            print('Backward fail (did not find any eigenvalue with eigs)')
            #e, v, w = lreig(tmpA) #fall back to solving full eig problem
            delta = np.zeros(weights.shape)


    if w.shape[0] != 0:
        divisor = w.T.dot(v).flatten()
        if abs(divisor) == 0:
            delta = np.zeros(weights.shape)
            print('Empty eig')
        else:
            delta = np.multiply(w[networkList[0]], v[networkList[1]])/divisor
            direction = e/np.abs(e)
            delta = (delta/direction).real
    else:
        #print('Empty eig')
        delta = np.zeros(weights.shape)

    constrainNorm = True
    if constrainNorm:
        norm = np.linalg.norm(delta)
        if norm>10:
            delta = delta/norm #typical seems to be ~0.36
        #delta = delta * numpy.abs(ctx.weights)
        #delta = delta/norm(delta)

    tangent_out = jnp.dot(delta,w_dot)

    return primal_out, tangent_out


In [19]:
#@jax.jit
def spectralLoss(model, YhatFull, expFactor = 20, lb=0.5, spectralTarget =0.95):
    """ spectral loss of the transmission function:
    
    check Fig3, panel A
    derivativeOfActivation(steadystate) * A 
    """

    # selects a random condition to be used in the spectral computation:
    randomIndex = np.random.randint(YhatFull.shape[0])

    # evaluate the derivative of the activation function at the steady state:
    activationFactor = oneStepDeltaActivationFactor(YhatFull[randomIndex,:], model.layers[1].leak)
    weightFactor = activationFactor.flatten()[model.layers[1].networkList[0]]
    multipliedWeightFactor =  model.layers[1].weights * weightFactor
    
    M = jnp.zeros(shape=model.layers[1].A.shape)
    ind = model.layers[1].networkList
    
    spectralRadius = getspectralRadius(multipliedWeightFactor, ind, M)
    
    scaleFactor = 1/jnp.exp(expFactor *  spectralTarget)
    
    #spectralRadiusLoss = scaleFactor * spectralRadius
    #spectralRadiusLoss = scaleFactor * (jnp.exp(expFactor*spectralRadius)-1)

    # https://jax.readthedocs.io/en/latest/faq.html#gradients-contain-nan-where-using-where
    spectralRadiusLoss = jnp.where(spectralRadius > lb, 
                                   scaleFactor * (jnp.exp(expFactor*spectralRadius)-1),
                                   jnp.zeros(1))[0]
    # if spectralRadius>lb:
    #     spectralRadiusLoss = scaleFactor * (jnp.exp(expFactor*spectralRadius)-1)
    # else:
    #     spectralRadiusLoss = 0.0

    return spectralRadiusLoss

In [20]:
# Test the spectral loss function
# iter_data = dataIndexloader((X, Y), batch_size, key=loader_key)
# dataIndex = next(iter_data)
# x = X[dataIndex, :]
# y = Y[dataIndex, :]
# M = jnp.zeros(shape = Model.model.layers[1].A.shape)

# (pred_y, pred_yFull) = jax.vmap(Model.model, out_axes=(0,0))(x)

# spectralLoss(Model.model, pred_yFull, expFactor = 21,spectralTarget = Model.trainingParameters.spectralTarget)

In [21]:
# v,g = jax.value_and_grad(spectralLoss)(Model.model, pred_yFull, expFactor = 21,spectralTarget =Model.trainingParameters.spectralTarget)
# print(v)
# print(g.layers[1].weights[1:20])
# g.layers[1].weights.max()

## Put together all the loss functions

In [22]:

#Setup optimizer
MoAFactor = 0.1
L2beta = 1e-8
spectralFactor = 1e-3
ligandFactor = 1e-5 #0.1 in toy
stateLossFactor = 1e-4

@jax.jit
def criterion(pred_y, y):
    
    return jnp.mean((y - pred_y) ** 2)

#@jax.jit
def criterion_mat(pred_y, y):
    # another version, can be called with different types of input.
    return jnp.mean((y - pred_y) ** 2)

@jax.jit
def simulate(model,x):
    return jax.vmap(model, out_axes=(0,0))(x)

filter_spec = jtu.tree_map(lambda _: False, Model.model)
filter_spec = eqx.tree_at(
    lambda tree: (tree.layers[1].weights, tree.layers[1].biases),
    filter_spec,
    replace=(True, True),
)

# jax/optax alternative: 
#! @eqx.filter_jit
@ft.partial(eqx.filter_value_and_grad, arg=filter_spec)
# @eqx.filter_value_and_grad
def compute_loss(model, x, y, dataIndex, currentState):
    
    (pred_y, pred_yFull) = simulate(model,x)

    currentState = currentState.at[dataIndex,1,:].set(jnp.squeeze(pred_yFull))

    # Avlant adds random noise between input layer and the recurrent layer. 
    fitLoss = criterion(pred_y,y)

    # signConstraints
    violated_weights = model.layers[1].getViolations(model.layers[1].weights)
    signConstraint = MoAFactor * jnp.sum(jnp.abs(jnp.where(violated_weights,model.layers[1].weights,0)))
    
    # ligand constraints: 
    ligandConstraint = ligandFactor * jnp.sum(jnp.square(model.layers[1].biases[model.layers[0].inOutIndices,0]))
    
    # State constraints: 
    stateLoss = stateLossFactor * uniformLoss(currentState, dataIndex, pred_yFull, maxConstraintFactor = 50)

    # Parameter costraints: 
    biasLoss = L2beta * jnp.sum(jnp.square(model.layers[1].biases))
    weightLoss = L2beta * jnp.sum(jnp.square(model.layers[1].weights))
    
    spectralRadiusLoss = spectralFactor * spectralLoss(model, pred_yFull, expFactor = 21)

    # sum of constraints:
    # loss = fitLoss + signConstraint + ligandConstraint + weightLoss + biasLoss + spectralFactor * spectralRadiusLoss + stateLoss
    loss = fitLoss + signConstraint + ligandConstraint + weightLoss + biasLoss + stateLoss + spectralRadiusLoss 

    return loss




In [23]:
#@ft.partial(jax.jit, static_argnums=1)

#g = eqx.filter_value_and_grad(compute_loss1,has_aux=True)
#fg = ft.partial(g,arg=filter_spec)


#@ft.partial(eqx.filter_value_and_grad, arg=filter_spec, has_aux=True)
def compute_loss1(model, x, y,  dataIndex, currentState):
    
    (pred_y, pred_yFull) = simulate(model,x)
    #model = eqx.combine(params, static)
    currentState = currentState.at[dataIndex,1,:].set(jnp.squeeze(pred_yFull))

    # Avlant adds random noise between input layer and the recurrent layer. 
    fitLoss = criterion(pred_y,y)

    # signConstraints
    violated_weights = model.layers[1].getViolations(model.layers[1].weights)
    signConstraint = MoAFactor * jnp.sum(jnp.abs(jnp.where(violated_weights,model.layers[1].weights,0)))
    
    # ligand constraints: 
    ligandConstraint = ligandFactor * jnp.sum(jnp.square(model.layers[1].biases[model.layers[0].inOutIndices,0]))
    
    # State constraints: 
    stateLoss = stateLossFactor * uniformLoss(currentState, dataIndex, pred_yFull, maxConstraintFactor = 50)

    # Parameter costraints: 
    biasLoss = L2beta * jnp.sum(jnp.square(model.layers[1].biases))
    weightLoss = L2beta * jnp.sum(jnp.square(model.layers[1].weights))
    
    # sum of constraints:
    # loss = fitLoss + signConstraint + ligandConstraint + weightLoss + biasLoss + spectralFactor * spectralRadiusLoss + stateLoss
    loss = fitLoss + signConstraint + ligandConstraint + weightLoss + biasLoss + stateLoss 

    return loss, pred_yFull

dec = ft.partial(eqx.filter_value_and_grad,arg=filter_spec,has_aux=True)
#dec_compute_loss1 = jax.jit(dec(compute_loss1))
dec_compute_loss1 = dec(compute_loss1)


@ft.partial(eqx.filter_value_and_grad, arg=filter_spec)
# @eqx.filter_value_and_grad
def compute_loss2(model, pred_yFull):
    
    spectralRadiusLoss = spectralFactor * spectralLoss(model, pred_yFull, expFactor = 21)

    return spectralRadiusLoss


In [28]:
def make_step(model, x, y, opt_state, dataIndex, currentState):
    
    # model = eqx.combine(params, static)
    #(pred_y, pred_yFull) = simulate(model,x)
    

    (loss1, pred_yFull), grads1 = dec_compute_loss1(model, x, y, dataIndex, currentState)
    loss2, grads2 = compute_loss2(model, pred_yFull)
    
    # detect if there is nan value in the gradients:
    if jnp.any(jnp.isnan(grads1.layers[1].weights)):
        
        num_of_nans = jnp.sum(jnp.isnan(grads1.layers[1].weights))
        print("nan in gradients of loss1: " )
        print(num_of_nans)
        raise Exception("nan in gradients of loss1")

    if jnp.any(jnp.isnan(grads2.layers[1].weights)):
        num_of_nans = jnp.sum(jnp.isnan(grads2.layers[1].weights))
        print("nan in gradients of loss2: ")
        print(num_of_nans)
        raise Exception("nan in gradients of loss2")

    #print("grads:")
    #print(grads.layers[1].weights)
    new_grads = jax.tree_map(lambda x, y: x+y, grads1, grads2)
    new_loss = loss1 + loss2

    #loss_old, grads_old = compute_loss(model, x, y, dataIndex, currentState)

    # print("loss diff:")
    # print(loss - loss_old)

    # print("grads diff:")
    # print(jnp.linalg.norm(grads.layers[1].weights - grads_old.layers[1].weights)/jnp.linalg.norm(grads_old.layers[1].weights))


    updates, opt_state = optim.update(new_grads, opt_state)
    model = eqx.apply_updates(model, updates)
    return new_loss, model, opt_state, currentState

In [25]:
#compute_loss_vg = jax.value_and_grad(compute_loss,has_aux=True)

# we have to use the partitioning trick for the make_step
#! @ft.partial(jax.jit, static_argnums=1)
def make_step_old(model, x, y, opt_state, dataIndex, currentState):
    
    loss, grads = compute_loss(model, x, y, dataIndex, currentState)
    
    #print("grads:")
    #print(grads.layers[1].weights)
    
    updates, opt_state = optim.update(grads, opt_state)
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state, currentState


# Whole optimization

In [26]:
def oneCycle(e, maxIter, maxHeight = 1e-3, startHeight=1e-5, endHeight=1e-5, minHeight = 1e-7, peak = 1000):
    phaseLength = 0.95 * maxIter
    if e<=peak:
        effectiveE = e/peak
        lr = (maxHeight-startHeight) * 0.5 * (np.cos(np.pi*(effectiveE+1))+1) + startHeight
    elif e<=phaseLength:
        effectiveE = (e-peak)/(phaseLength-peak)
        lr = (maxHeight-endHeight) * 0.5 * (np.cos(np.pi*(effectiveE+2))+1) + endHeight
    else:
        lr = endHeight
    return lr

In [29]:
optim = optax.inject_hyperparams(optax.adam)(learning_rate=1)
opt_state = optim.init(Model.model)

resetState = opt_state # state is a tuple, copy it by assignmment
# params, static = eqx.partition(Model.model, eqx.is_array)
model = Model.model
maxIter = 1000
batch_size = 5
loader_key = jax.random.PRNGKey(142)
loader_key, statekey = jax.random.split(loader_key)

progress = nn_models.rnnModel.OptimProgress(maxIter)

currentState = jax.random.uniform(shape=(N, 1, Model.model.layers[1].biases.shape[0]),key=statekey)

for e in range(maxIter):

    currentLoss = []
    learning_rate_e = oneCycle(e, maxIter, maxHeight = 2e-3, minHeight = 1e-8, peak = 1000)
    opt_state.hyperparams['learning_rate'] = learning_rate_e
    
    curLoss = []
    curEig = []
    iter_data = dataIndexloader((X, Y), batch_size, key=loader_key)

    for dataIndex in iter_data:
        x = X[dataIndex, :]
        y = Y[dataIndex, :]

        loss, model, opt_state, currentState = make_step(model, x, y, opt_state, dataIndex, currentState)
        
        currentLoss.append(loss.item())

    #fitLoss = compute_test(params, static, Xtest, Ytest)
    trained_model = model #eqx.combine(params, static)
    

    # compute progress on the validation set:     
    #Yhat = jax.vmap(trained_model)(Xtest)
    #fitLoss = criterion_mat(Yhat,Ytest)
    
    progress.stats['violations'][e] = np.sum(trained_model.layers[1].getViolations()).item()
    #progress.stats['test'][e] = fitLoss.item()
    progress.storeProgress(loss=currentLoss, lr=learning_rate_e, violations=np.sum(trained_model.layers[1].getViolations(trained_model.layers[1].weights)).item())

    if e % 100 == 0:
        progress.printStats(e)

    if np.logical_and(e % 100 == 0, e>0):
        opt_state = resetState


# pytorch version finished 10 000 iteration in 7:52 mins
# i=9900, l=0.00012, s=0.718, r=0.00001, v=0

i=0, l=0.20176, r=0.00001, v=0
i=100, l=0.19464, r=0.00006, v=0
i=200, l=0.15572, r=0.00020, v=0
i=300, l=0.08540, r=0.00042, v=0
i=400, l=0.05925, r=0.00070, v=0
i=500, l=0.05085, r=0.00100, v=0
i=600, l=0.03634, r=0.00131, v=0
Backward fail (eigs left returned different eigenvalue)
i=700, l=0.02174, r=0.00159, v=1
i=800, l=0.01603, r=0.00181, v=1
i=900, l=0.01218, r=0.00195, v=1


In [51]:
M = np.loadtxt("M.csv", delimiter=",")
net = np.loadtxt("ind.csv", delimiter=",").astype(int)
w = np.loadtxt("w.csv", delimiter=",")

In [52]:
M[net[0],net[1]] = w

In [55]:
np.sum(np.abs(M),axis=1).max()

3.775542013347149

In [58]:
e,v = np.linalg.eig(M)
max(abs(e))

0.37025019316109226